# Multi-hospital Organ Transplants

This notebook explores expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent directory, **MultipleAgents**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

<font color='red'>IMPORTANT NOTE: you should run **AgentsAndEvents.ipynb** prior to running this notebook to ensure that **AE.pl** contains up to date predicates! You must run the notebook, not just update and save it.</font>

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

## Compatibility / Suitability

An agent may expect that when it requests a resource, and its request is matched to a resource offer from another agent, that the match is "suitable". In the organ transplant scenario, the recipient's hospital might expect that any organ offered to the recipient by the monitoring system is compatible with the recipient in terms of blood type, medical conditions and any other important factors. Similarly, the donor's hospital might expect that the recipient of their offered organ is a suitable recipient, such that the organ is not wasted. 

### Compatibility rules

In order to simplify predicate rules, we declare a helper function which allows us to either select a singular argument or get arguments from a list

In [2]:
% File: extract.pl
extract(Input, Result):- is_list(Input), member(Result, Input).
extract(Input, Input):- \+ is_list(Input).

We can declare compatibility rules for matching an organ donor to a recipient. The *newCompatRule* is used to initiate new compatibility rules, while *newIncompatRule* initiates new incompatibility rules. Each terminates the opposing compatibility rule. This may require some tweaking if we wish opposing rules to cause a conflict violation rather than overwriting an earlier rule.

For donor-patient matching, we wish to differentiate between known compatibility or incompatibility and unknown compatibility. For instance, if we know that severe asthma makes lungs unsuitable for organ donation, but does not affect other organs, the compatibility of organs from a patient with severe asthma to a potential recipient should be treated differently to the compatibility of organs from a patient with Parkinson's disease, if the effect of this condition is not defined by any existing compatibility rules.

Compatibility rules have the arguments *Type*, *Donor* and *Recipient*. *Type* refers to the type of compatibility rule - currently defined are *bloodType*, *disease*, and *ageOrgan*. *bloodType* involves comparing the blood type of the donor and recipient, *disease* the medical conditions of the two parties, and *ageOrgan* controls the compatibility between a donor's organ type and their age. *Donor* is generally made up of a singular term, while *Recipient* can be either a singular term or a list. Having *Recipient* as a list makes it easier to encode certain compatibility rules, particularly those concerning blood types.

For example, *compatRule(bloodType, "B", ["B", "AB"])* states that if the donor has blood type B, their organs can be donated to patients who have blood type B or AB. As the four types of blood are initially declared to be A, B, AB and O, we can then infer that such a donor cannot donate to patients with blood type A or O.

As our compatibility rules may change over time, they are encoded as fluents. An example in which compatibility rules change over time, is the suitability of organs from donors with Hepatitis B and C or HIV. In the past, these organs were not suitable for donation (except potentially to recipients already diagnosed with the disease). However, a recent study has shown that if recipients are given antiviral drugs, this can prevent them from getting the disease from an infected organ. 

(See https://www.npr.org/sections/health-shots/2019/04/03/709533047/hepatitis-c-not-a-barrier-for-organ-transplantation-study-finds.)

Before such a study is recognised by a medical institution, the hospital may have a compatibility rule stating that Hepatitis B, Hepatitis C and HIV patients cannot donate to recipients without the disease(s). Once such a study is approved, the hospital may wish to change the compatibility rule to state that such donations are acceptable provided that an appropriate antiviral drug treatment is undertaken by the recipient.

All of our agents may make use of compatibility rules (although not necessarily the same ones), so we declare our *initiates* and *terminates* clauses irrespective of the agent.

In [3]:
% File: compatabilityRules.pl
initiates(_:newCompatRule(Type, Donor, Recipient), compatRule(Type, Donor, Recipient),_).
terminates(_:newIncompatRule(Type, Donor, Recipient), compatRule(Type, Donor, Recipient),_).
initiates(_:newIncompatRule(Type, Donor, Recipient), incompatRule(Type, Donor, Recipient),_).
terminates(_:newCompatRule(Type, Donor, Recipient), incompatRule(Type, Donor, Recipient),_).

Currently, compatibility rules have been declared for organs, diseases, blood types, ages, and combinations of these categories. The organs used and age compatibility rules are based on information provided by *Organ Donation New Zealand*. 

(See https://www.donor.co.nz/facts-and-myths/.)

Blood type compatibility is based on information from *UC Davis Health*. 

(See https://health.ucdavis.edu/transplant/livingkidneydonation/matching-and-compatibility.html.)

We set our initial compatibility rules to apply for *monSys*, as it can check whether a donor and a recipient are compatible when a match occurs. We declare *organs*, *diseases*, and *bloodTypes* fluents so that we are able to ascertain which category a particular property of a patient belongs to. While it may be unlikely that any new blood types are about to be discovered, given that the existing categories were determined over a century ago, it is best practice to design a system that can cope with changes if necessary. A possible way to achieve a new blood type may be a mutation, or widening our organ donation system to animal organ transplants.

The *ageOrgan* compatibility rules include an organ type, and either a two argument list or the argument *all*. *all* is interpreted to mean that donors of all ages can donate an organ of this type. If a list is used, the first value is the minimum age of the donor and the second value is the maximum age.

We also declare a disease compatibility rule stating that a donor and recipient are compatible if the donor and the recipient have the same disease (and the donor has no other diseases not present in the recipient).

In [4]:
% File: initialRules.pl
initially(monSys:organs(["Heart","Kidney", "Liver","Lungs", "Pancreas"])).
initially(monSys:diseases(["HIV", "Hep-B", "Hep-C", "Cancer", "Severe Asthma", "Chronic Pulmonary Disease"])).
initially(monSys:bloodTypes(["A", "AB", "B", "O"])).

initially(monSys:compatRule(bloodType, "A", ["A", "AB"])).
initially(monSys:compatRule(bloodType, "B", ["B", "AB"])).
initially(monSys:compatRule(bloodType, "AB", ["AB"])).
initially(monSys:compatRule(bloodType, "O",["A","AB","B","O"])).
initially(monSys:compatRule(ageOrgan, "Heart", [0, 65])).
initially(monSys:compatRule(ageOrgan, "Lungs", [0, 70])).
initially(monSys:compatRule(ageOrgan, "Liver", all)).
initially(monSys:compatRule(ageOrgan, "Pancreas", [0, 45])).
initially(monSys:compatRule(ageOrgan, "Kidney", all)).
initially(monSys:compatRule(disease,X, X)).
initially(monSys:incompatRule(diseaseOrgan, "Heart Disease", "Liver")).

From our compatibility rules, we wish to establish resulting compatibility fluents. These fluents do not have initiate or terminate clauses associated with them, and are based on the current compatibility rules.

In [5]:
% File: resultingCompatibilities.pl

% The donor and recipient blood types are compatible if the recipient blood type is a member of the recipient list for the donor compatibility rule.
holdsAt(Agent, compat(bloodType, X, Y),T):- holdsAt(Agent, bloodTypes(BL),T), member(Y, BL), member(X, BL), holdsAt(Agent, compatRule(bloodType, L1, L2),T), 
extract(L1,X), extract(L2, Y).
% Otherwise, they are incompatible.
holdsAt(Agent, incompat(bloodType, X, Y),T):- \+ holdsAt(Agent, compat(bloodType, X, Y),T).

% A donor is within the acceptable age range for donating a particular organ if their age is between the minimum and maximum age specified,
% or if donors of all ages are eligible.
holdsAt(Agent, compat(ageOrgan, Disease, Age),T):- holdsAt(Agent, compatRule(ageOrgan, Diseases, Range),T), (Range = all; (Range = [Min, Max], Age =< Min, 
Age >= Max)), extract(Diseases, Disease).
% Otherwise, they are not eligible.
holdsAt(Agent, incompat(ageOrgan, Disease, Age),T):- \+ holdsAt(Agent, compat(ageOrgan, Disease, Age),T).

% General rule to status
holdsAt(Agent, incompat(Type, X, Y),T):- holdsAt(Agent, incompatRule(Type, L1, L2),T), extract(L1,X), extract(L2,Y), Type \= bloodType, Type \= ageOrgan.
holdsAt(Agent, compat(Type, X, Y),T):- holdsAt(Agent, compatRule(Type, L1, L2),T), extract(L1, X), extract(L2,Y), Type \= bloodType, Type \= ageOrgan.


In [6]:
happensAtNarrative("Otago":send(monSys, waitAddReq(104, ["heart", "lung"], [bloodType:"B", dob:"01/04/2015", conditions:["Asthma"]])),0).

happensAtNarrative("Wellington": send(monSys, donorOffer(201, [heart, kidney, liver, lungs, pancreas], [bloodType:"A", dob:"10/11/1978", conditions:["HepC"]])),3).
happensAtNarrative(monSys:match(701, "Otago", 104, "Wellington", 201, "Auckland", ["heart"], [donorCondition:"HepC"]),4).

In [7]:
?- run(10).
?- happensAt(monSys, Event, T).

true.
Event = match(701, b'Otago', 104, b'Wellington', 201, b'Auckland', [b'heart'], [Functor(188685,2,donorCondition,b'HepC')]), T = 4 ;
Event = receive(b'Otago', waitAddReq(104, [b'heart', b'lung'], [Functor(188685,2,bloodType,b'B'), Functor(188685,2,dob,b'01/04/2015'), Functor(188685,2,conditions,[b'Asthma'])])), T = 1 ;
Event = receive(b'Wellington', donorOffer(201, [Atom('679941'), Atom('680069'), Atom('680197'), Atom('680325'), Atom('680453')], [Functor(188685,2,bloodType,b'A'), Functor(188685,2,dob,b'10/11/1978'), Functor(188685,2,conditions,[b'HepC'])])), T = 4 ;
Event = send(b'Otago', waitAccept(104)), T = 2 ;
Event = send(b'Otago', recipientFound(701, 104, b'Wellington', b'Auckland', [b'heart'], [Functor(188685,2,donorCondition,b'HepC')])), T = 5 ;
Event = send(b'Wellington', donorFound(701, 201, b'Otago', b'Auckland', [b'heart'], [Functor(188685,2,donorCondition,b'HepC')])), T = 5 ;
Event = send(b'Auckland', locationSelected(701, b'Otago', b'Wellington', [b'heart'], [Functor

In [8]:
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(ID, Organs, _))), 
within(or([happ(noMatch(Hospital, ID)), happ(match(_, Hospital, ID, _, _, _, Organs, _))]),1), 
dependent, "Timely response":"A donation offer should be matched or refused in the next time period"), _).

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>